# Update nba datasets

## Setup and imports

In [ ]:
import os
import jsonlines
import pandas as pd
import sqlite3

In [ ]:
pd.options.display.max_columns = 99
pd.options.display.max_rows = 99

## Read datasets from jsonlines files

In [ ]:
most_recent = max(os.listdir("../data"))

In [ ]:
seasons_df = pd.read_json(f"../data/{most_recent}/season.jl", lines=True)
games_df = pd.read_json(f"../data/{most_recent}/game.jl", lines=True)
players_df = pd.read_json(f"../data/{most_recent}/player.jl", lines=True)

In [ ]:
boxscores_file = f"../data/{most_recent}/boxscore.jl"
with jsonlines.open(boxscores_file) as reader:
    basic_boxscores_df = pd.DataFrame(
        list(filter(lambda x: x.get('boxscore')=='basic', iter(reader)))
    )

with jsonlines.open(boxscores_file) as reader:
    advanced_boxscores_df = pd.DataFrame(
        list(filter(lambda x: x.get('boxscore')=='advanced', iter(reader)))
    )

## Clean data

Type conversions

In [ ]:
def convert_dtypes(data):
    for col in data:
        try:
            data[col] = data[col].astype(float)
        except ValueError:
            pass
        except TypeError:
            pass
    return data

In [ ]:
basic_boxscores_df = convert_dtypes(basic_boxscores_df)
advanced_boxscores_df = convert_dtypes(advanced_boxscores_df)
games_df = convert_dtypes(games_df)
seasons_df = convert_dtypes(seasons_df)
players_df = convert_dtypes(players_df)

Fix links

In [ ]:
def fix_link(data, column):
    root = "https://www.basketball-reference.com"
    data[column] = root + data[column].str.replace(root, "")
    return data

In [ ]:
basic_boxscores_df = basic_boxscores_df.pipe(fix_link, "player_link")
advanced_boxscores_df = advanced_boxscores_df.pipe(fix_link, "player_link")
seasons_df = seasons_df.pipe(fix_link, "season_link")
players_df = players_df.pipe(fix_link, "player_link")
games_df = (
    games_df
        .pipe(fix_link, "boxscore_link")
        .pipe(fix_link, "home_link")
        .pipe(fix_link, "visitor_link")
)
basic_boxscores_df = basic_boxscores_df.rename(columns={"game_url": "boxscore_link"})
advanced_boxscores_df = advanced_boxscores_df.rename(columns={"game_url": "boxscore_link"})

Update mp to fraction

In [ ]:
def minutes_string_to_float(series):
    return (
        series
         .str.split(":", expand=True)
         .apply(lambda x: x.astype(float))
         .rename(columns = {0: "mins", 1: "secs"})
         .eval('mins + secs/60')
    )

In [ ]:
basic_boxscores_df['minutes'] = minutes_string_to_float(basic_boxscores_df.mp)
advanced_boxscores_df['minutes'] = minutes_string_to_float(advanced_boxscores_df.mp)

Stack home and away games data

In [ ]:
home_games = games_df[
    ['attendance', 'boxscore_link', 'date', 'home', 'home_link', 'home_pts',
     'visitor', 'visitor_link', 'visitor_pts',
     'notes', 'overtime', 'season_link', 'start'
    ]].copy()
home_games['location'] = 'home'
home_games = home_games.rename(columns={'home': 'team', 'home_link': 'team_link', 'home_pts': 'pts', 'visitor': 'opp', 'visitor_link': 'opp_link', 'visitor_pts': 'opp_pts'})

visitor_games = games_df[
    ['attendance', 'boxscore_link', 'date', 'visitor', 'visitor_link',
     'visitor_pts', 'home', 'home_link', 'home_pts', 
     'notes', 'overtime', 'season_link', 'start'
    ]].copy()
visitor_games['location'] = 'away'
visitor_games = visitor_games.rename(columns={'visitor': 'team', 'visitor_link': 'team_link', 'visitor_pts': 'pts', 'home': 'opp', 'home_link': 'opp_link', 'home_pts': 'opp_pts'})

In [ ]:
normalized_games_df = pd.concat([home_games, visitor_games], ignore_index=True)

Identify playoff games

In [ ]:
def game_number(data):
    return (
        data
        .groupby(['season_link', 'team'])
        ['date'].rank()
    )

def team_game_count(data):
    return (
        data
        .eval("counter = 1")
        .groupby(['season_link', 'team'])
        ['counter'].transform('count')
    )

def min_team_games(data):
    return (
        data
        .groupby(['season_link'])
        ['team_game_count'].transform('min')
    )

In [ ]:
normalized_games_df['playoffs'] = (
    normalized_games_df
     .assign(game_number=game_number)
     .assign(team_game_count=team_game_count)
     .assign(min_team_games=min_team_games)
     .eval('game_number > min_team_games')
)

## Data quality checks

Confirm that games in basic boxscores matches games in game summaries

In [ ]:
games_df.groupby(["season_link"]).apply(lambda x: len(x.boxscore_link.unique()) == len(x.boxscore_link))

In [ ]:
games_df[~(games_df.boxscore_link).isin(basic_boxscores_df.boxscore_link.unique())]

Confirm that the number of points in boxscore match number of points in games

In [ ]:
total_points = (
    basic_boxscores_df
    .groupby('boxscore_link')['pts']
    .sum()
)

In [ ]:
total_points_game = (
    games_df
    .rename(columns={'boxscore_link': 'boxscore_link'})
    .assign(total_pts = lambda x: x.home_pts + x.visitor_pts)
    .merge(total_points.reset_index(),
        how='outer',
        on='boxscore_link'
    )
)

In [ ]:
(
    total_points_game
    .assign(mismatch = lambda x: x.total_pts - x.pts)
    .assign(abs_mismatch = lambda x: abs(x.mismatch))
    .groupby('season_link')['mismatch', 'abs_mismatch']
    .aggregate('mean')
    .query('abs_mismatch != 0')
)

Check total number of games in a season

In [ ]:
(
    normalized_games_df
    .groupby(['season_link', 'team'])['boxscore_link']
    .count()
    .groupby(['season_link'])
    .aggregate(["max", "min"])
    .eval("diff = max - min")
)

## Load current data

In [ ]:
conn = sqlite3.connect('../db/nbafantasy.db')

In [ ]:
seasons_df.to_sql("seasons", con=conn, if_exists="replace", index=False)
players_df.to_sql("players", con=conn, if_exists="replace", index=False)

In [ ]:
normalized_games_df.to_sql("games_update", con=conn, if_exists="replace", index=False)
basic_boxscores_df.to_sql("basic_boxscore_update", con=conn, if_exists="replace", index=False)
advanced_boxscores_df.to_sql("adv_boxscore_update", con=conn, if_exists="replace", index=False)

In [ ]:
cur = conn.cursor()
cur.execute(
    """
    delete from games
    where (date, team_link, opp_link, season_link) in (select date, team_link, opp_link, season_link from games_update)
    """
)
cur.execute(
    """
    insert into games
    select *
    from games_update
    """
)

In [ ]:
cur = conn.cursor()
cur.execute(
    """
    delete from basic_boxscore
    where (boxscore_link, player_link) in (select boxscore_link, player_link from basic_boxscore_update)
    """
)
cur.execute(
    """
    insert into basic_boxscore
    select *
    from basic_boxscore_update
    """
)

In [ ]:
cur = conn.cursor()
cur.execute(
    """
    delete from adv_boxscore
    where (boxscore_link, player_link) in (select boxscore_link, player_link from adv_boxscore_update)
    """
)
cur.execute(
    """
    insert into adv_boxscore
    select *
    from adv_boxscore_update
    """
)